In [1]:
import sys, os
sys.path.insert(0, '..')

import numpy as np
import scipy.sparse as sp
import torch
import networkx as nx
import pickle as pkl
from utils.utils import sparse_to_torch_tensor, normalize_row_wise
from utils import score
from utils.config import *

In [2]:

t = 'fairwalk'
embs = pkl.load(open('/tmp/embs.pkl', 'rb'))
rgraphs = pkl.load(open("/tmp/rgraphs.pkl", 'rb'))
labels = torch.LongTensor(pkl.load(open("/tmp/group_ids.pkl", 'rb')))
adj = nx.to_scipy_sparse_matrix(rgraphs[t])
adj = sparse_to_torch_tensor(normalize_row_wise(adj + sp.eye(adj.shape[0])))
features = torch.FloatTensor(embs[t])
idx_train, idx_val = torch.LongTensor(range(80)), torch.LongTensor(range(80, 105))
features.shape, type(features), labels.shape, type(labels), adj.shape, type(adj)

(torch.Size([105, 128]),
 torch.Tensor,
 torch.Size([105]),
 torch.Tensor,
 torch.Size([105, 105]),
 torch.Tensor)

In [3]:
import math
import torch

from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
from utils.node_classification import GCN, GAT


In [4]:
import torch.nn as nn
import torch.nn.functional as F

In [5]:
import time
import argparse
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim


# Training hyperparameter configuration
class Args:
    no_cuda = False # Whether to use cuda/gpu
    seed = 42 # Set random seed
    epochs = 200 # number of iterations
    lr = 0.01 # learning rate
    weight_decay = 5e-4 # Learning rate decay
    n_hid = 16 # hidden layer dimension
    dropout = 0.5 # dropout rate

In [6]:
input_features = features.shape[1]
models = {}
models['GCN'] = GCN(n_feat=input_features,
            n_hids=[128,32],
            n_classes=labels.max().item() + 1)
models['GAT'] = GAT(n_feat=input_features,
            n_hids=8,
            n_classes=labels.max().item() + 1,
                   nheads=3)

In [7]:
args = Args()

def train_model(model):
    optimizer = optim.Adam(model.parameters(),
                       lr=args.lr, 
                       weight_decay=args.weight_decay)
    
    return model.fit(X=features, y=labels, optimizer=optimizer, adj_list=adj, log=False) 

In [8]:
# Model test function
def test(model, X, y, adj):
    # First mark the model as eval mode
    model.eval()
    output = model(X, adj)

    loss_test = F.nll_loss(output, y)
    acc_test = score.accuracy(output, y)
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))

In [9]:
for key, model in models.items(): 
    train_model(model)
    print(test(model, features, labels, adj))

Test set results: loss= 0.1948 accuracy= 0.8857
None
Test set results: loss= 0.2345 accuracy= 0.8952
None
